In [1]:
import re
import copy
import pandas as pd
import numpy as np

In [2]:
columns = ["prev", "curr", "type", "n"]
de_wiki = pd.read_csv("../data/22/clickstream-dewiki-2022-03.tsv", delim_whitespace=True, names=columns, quoting=3)
pl_wiki = pd.read_csv("../data/22/clickstream-plwiki-2022-03.tsv", delim_whitespace=True, names=columns, quoting=3)
en_wiki = pd.read_csv("../data/22/clickstream-enwiki-2022-03.tsv", delim_whitespace=True, names=columns, quoting=3)

In [4]:
badges = pd.read_xml("../data/22/Badges.xml")
users = pd.read_xml("../data/22/Users.xml")

In [5]:
posts = pd.read_xml("../data/22/Posts.xml")
votes = pd.read_xml("../data/22/Votes.xml")

In [6]:
tags = pd.read_xml("../data/22/Tags.xml")

In [18]:
de_wiki.head(5)

,prev,curr,type,n
0,other-empty,Lichtröhre,external,53
1,Lichtleiter,Lichtröhre,link,12
2,Wohnhöhle,Lichtröhre,link,19
3,other-search,Lichtröhre,external,76
4,other-internal,Lichtröhre,external,11


**Na jaki artykuł najczęściej wchodzili w marcu użytkownicy Wikipedii niemieckiej?**

In [20]:
de_wiki.groupby("curr")['n'].sum().sort_values(ascending=False).head()

curr
Ukraine                                     1979272
Wladimir_Wladimirowitsch_Putin              1548451
Oligarch                                    1322517
Russischer_Überfall_auf_die_Ukraine_2022    1109265
Wolodymyr_Selenskyj                         1051957
Name: n, dtype: int64

**Na jaki artykuł najczęściej wchodzili w marcu użytkownicy Wikipedii polskiej?**

In [21]:
pl_wiki.groupby("curr")['n'].sum().sort_values(ascending=False).head()

curr
Ukraina                            944630
Władimir_Putin                     698327
Wołodymyr_Zełenski                 643374
NATO                               592215
Inwazja_Rosji_na_Ukrainę_(2022)    468117
Name: n, dtype: int64

**Jaka jest DisplayName użytkownika który zebrał najwięcej badge'y na Travel.StackExchange?**

In [26]:
badges

,Id,UserId,Name,Date,Class,TagBased
0,1,2,Autobiographer,2011-06-21T20:16:48.910,3,False
1,2,3,Autobiographer,2011-06-21T20:16:48.910,3,False
2,3,4,Autobiographer,2011-06-21T20:16:48.910,3,False
3,4,6,Autobiographer,2011-06-21T20:21:49.157,3,False
4,5,8,Autobiographer,2011-06-21T20:21:49.157,3,False
...,...,...,...,...,...,...
198770,230052,12046,Yearling,2022-03-05T17:51:13.357,2,False
198771,230053,127069,Autobiographer,2022-03-05T18:35:25.153,3,False
198772,230054,4801,Popular Question,2022-03-05T21:02:40.993,3,False
198773,230055,127048,Nice Question,2022-03-05T21:37:27.870,3,False


In [109]:
max_user_badges = badges["UserId"].value_counts().reset_index(name="cnt").rename(columns={"index": "UserId"})
max_user_badges

,UserId,cnt
0,101,2202
1,1893,1197
2,9009,1037
3,140,894
4,10051,752
...,...,...
58310,10199,1
58311,68510,1
58312,16446,1
58313,16453,1


In [112]:
users.loc[users["Id"] == max_user_badges.iloc[0,:]["UserId"], "DisplayName"]

97    Mark Mayo
Name: DisplayName, dtype: object

(Kontynuacja powyższego) **Ile było wejść na stronę poświęconą miastu (nie całemu Location) z którego jest ten użytkownik na angielskiej Wikipedii?**

In [115]:
user_city = users.loc[users["Id"] == max_user_badges.iloc[0,:]["UserId"], "Location"].reset_index()
user_city

,index,Location
0,97,"Christchurch, New Zealand"


In [126]:
en_wiki.loc[en_wiki["curr"] == "Christchurch"]['n'].sum()

25804

**Czyj (DisplayName) post ma najwięcej UpVotes - DownVotes?**

In [136]:
votes.head()

,Id,PostId,VoteTypeId,CreationDate,UserId,BountyAmount
0,1,1,2,2011-06-21T00:00:00.000,NaN,NaN
1,2,1,2,2011-06-21T00:00:00.000,NaN,NaN
2,3,2,2,2011-06-21T00:00:00.000,NaN,NaN
3,4,3,2,2011-06-21T00:00:00.000,NaN,NaN
4,5,5,5,2011-06-21T00:00:00.000,13.0,NaN


In [137]:
up_vote_key = 2
down_vote_key = 3

In [151]:
posts_up_votes = votes.loc[votes["VoteTypeId"] == up_vote_key].groupby("PostId").size().reset_index(name="UpVotes")
posts_up_votes

,PostId,UpVotes
0,1,10
1,2,43
2,3,16
3,4,9
4,5,14
...,...,...
114396,172462,4
114397,172463,3
114398,172464,3
114399,172467,1


In [152]:
posts_down_votes = votes.loc[votes["VoteTypeId"] == down_vote_key].groupby("PostId").size().reset_index(name="DownVotes")
posts_down_votes

,PostId,DownVotes
0,1,2
1,3,1
2,4,1
3,7,3
4,8,1
...,...,...
37288,172448,4
37289,172451,1
37290,172456,5
37291,172459,3


In [155]:
merged_votes = posts_up_votes.merge(
    posts_down_votes,
    left_on="PostId",
    right_on="PostId",
)

In [181]:
merged_votes.head()


,PostId,OwnerUserId,UpVotes,DownVotes,Up-Down
0,1,9.0,10,2,8
1,3,9.0,16,1,15
2,4,24.0,9,1,8
3,8,26.0,16,1,15
4,9,23.0,15,2,13


In [157]:
merged_votes["Up-Down"] = merged_votes["UpVotes"] - merged_votes["DownVotes"]

In [158]:
merged_votes

,PostId,UpVotes,DownVotes,Up-Down
0,1,10,2,8
1,3,16,1,15
2,4,9,1,8
3,7,6,3,3
4,8,16,1,15
...,...,...,...,...
21121,172399,3,1,2
21122,172408,14,2,12
21123,172411,21,1,20
21124,172426,3,2,1


In [164]:
merged_votes = merged_votes.merge(
    posts,
    left_on = "PostId",
    right_on = "Id"
).loc[:, ["PostId", "OwnerUserId", "UpVotes", "DownVotes","Up-Down"]]
merged_votes

,PostId,OwnerUserId,UpVotes,DownVotes,Up-Down
0,1,9.0,10,2,8
1,3,9.0,16,1,15
2,4,24.0,9,1,8
3,8,26.0,16,1,15
4,9,23.0,15,2,13
...,...,...,...,...,...
16315,172399,75733.0,3,1,2
16316,172408,127027.0,14,2,12
16317,172411,3884.0,21,1,20
16318,172426,15281.0,3,2,1


In [188]:
merged_votes = merged_votes.sort_values("Up-Down", ascending=False)

In [189]:
users.loc[users["Id"] == merged_votes.iloc[0,:]["OwnerUserId"], "DisplayName"]

7474    Nean Der Thal
Name: DisplayName, dtype: object

**W jakim miesiącu oddano najwięcej głosów?**

In [217]:
votes['CreationDate'] = pd.to_datetime(votes['CreationDate'])

In [218]:
votes.head()

,Id,PostId,VoteTypeId,CreationDate,UserId,BountyAmount
0,1,1,2,2011-06-21,NaN,NaN
1,2,1,2,2011-06-21,NaN,NaN
2,3,2,2,2011-06-21,NaN,NaN
3,4,3,2,2011-06-21,NaN,NaN
4,5,5,5,2011-06-21,13.0,NaN


In [259]:
date_votes = copy.deepcopy(votes)

In [260]:
date_votes["CreationDate"] = date_votes['CreationDate'].apply(lambda x: x.strftime('%Y-%m'))

In [261]:
date_votes = date_votes["CreationDate"].value_counts().reset_index(name="ctn").rename(columns={"index": "CreationDate"})

In [264]:
date_votes = date_votes.sort_values("CreationDate")

**Na przełomie jakich dwóch miesiecy nastąpił największy spadek ilości oddanych głosów?**

In [265]:
date_votes_start = copy.deepcopy(date_votes[:-1].reset_index())

In [266]:
date_votes_end = copy.deepcopy(date_votes[1:].reset_index())

In [268]:
date_votes_end

,index,CreationDate,ctn
0,107,2011-07,3388
1,118,2011-08,2922
2,104,2011-09,3551
3,84,2011-10,4358
4,82,2011-11,4543
...,...,...,...
124,96,2021-11,3850
125,95,2021-12,3886
126,108,2022-01,3369
127,124,2022-02,2660


In [270]:
summary_date = pd.DataFrame([date_votes_start["CreationDate"], date_votes_end["CreationDate"], date_votes_end["ctn"]-date_votes_start["ctn"]]).transpose()

In [271]:
summary_date.sort_values("ctn")

,CreationDate,CreationDate,ctn
51,2015-09,2015-10,-6201
69,2017-03,2017-04,-4081
99,2019-09,2019-10,-3607
67,2017-01,2017-02,-3594
64,2016-10,2016-11,-3561
...,...,...,...
84,2018-06,2018-07,3249
50,2015-08,2015-09,3290
34,2014-04,2014-05,3538
53,2015-11,2015-12,3557


**Jaki tag (patrzymy tylko na Posts) jest najpopularniejszy wśród użytkowników z Polski?**

In [288]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Tags,AnswerCount,CommentCount,ClosedDate,ContentLicense,FavoriteCount,ParentId,LastEditorDisplayName,CommunityOwnedDate,OwnerDisplayName
0,1,1,393.0,2011-06-21T20:19:34.730,8,589.0,<p>My fiancée and I are looking for a good Car...,9.0,101.0,2011-12-28T21:36:43.910,...,<caribbean><cruising><vacations>,4.0,4,2013-02-25T23:52:47.953,CC BY-SA 3.0,NaN,NaN,None,None,None
1,2,1,NaN,2011-06-21T20:22:33.760,43,3349.0,"<p>This was one of our definition questions, b...",13.0,19400.0,2021-04-27T00:47:20.470,...,<guides><extreme-tourism><amazon-river><amazon...,8.0,4,None,CC BY-SA 4.0,6.0,NaN,None,None,None
2,3,2,NaN,2011-06-21T20:24:28.080,15,NaN,<p>One way would be to go through an Adventure...,9.0,NaN,None,...,None,NaN,2,None,CC BY-SA 3.0,NaN,2.0,None,None,None
3,4,1,NaN,2011-06-21T20:24:57.160,8,292.0,<p>Singapore Airlines has an all-business clas...,24.0,693.0,2013-01-09T09:55:22.743,...,<loyalty-programs><routes><ewr><singapore-airl...,1.0,1,None,CC BY-SA 3.0,NaN,NaN,None,None,None
4,5,1,770.0,2011-06-21T20:25:56.787,14,479.0,<p>Another definition question that interested...,13.0,101.0,2011-12-28T21:36:18.230,...,<romania><transportation>,5.0,0,None,CC BY-SA 3.0,2.0,NaN,None,None,None


In [276]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,ProfileImageUrl
0,-1,1,2011-06-21T15:16:44.253,Community,2011-06-21T15:16:44.253,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",465,6309,24756,-1.0,None
1,2,101,2011-06-21T20:10:03.720,Geoff Dalgas,2017-10-09T14:25:03.613,http://stackoverflow.com,"Corvallis, OR",<p>Dev #2 who helped create Stack Overflow cur...,41,1,0,2.0,https://i.stack.imgur.com/nDllk.png?s=128&g=1
2,3,101,2011-06-21T20:11:02.490,Nick Craver,2019-11-13T23:54:04.630,https://nickcraver.com/blog/,"Winston-Salem, NC",<p>I am a Principal Software Engineer at Micro...,22,1,0,7598.0,https://i.stack.imgur.com/nGCYr.jpg?s=128&g=1
3,4,101,2011-06-21T20:12:21.223,Emmett,2019-09-05T02:57:12.997,http://minesweeperonline.com,"San Francisco, CA","<p>co-founder of <a href=""https://airtable.com...",16,1,0,1998.0,https://i.stack.imgur.com/d1oHX.jpg?s=128&g=1
4,5,101,2011-06-21T20:15:31.410,Kevin Montrose,2019-05-21T18:43:29.443,https://kevinmontrose.com,"New York, NY, United States","<p><a href=""http://blog.stackoverflow.com/2010...",14,3,0,29738.0,None


In [286]:
users_poland = users.loc[users["Location"].str.contains('Poland|Polska') == True, ["Id","Location"]]

In [287]:
users_poland

,Id,Location
42,44,"Warsaw, Poland"
331,355,"Gdansk, Poland"
1004,1074,Poland
1138,1214,"Gliwice, Silesian, Poland"
1145,1223,Poland
...,...,...
90190,124004,"Wroclaw, Poland"
90427,124249,"Warsaw, Poland"
90478,124302,Poland
90843,125684,"Warszawa, Polska"


In [291]:
posts_poland = posts.loc[posts["OwnerUserId"].isin(users_poland["Id"]), ["Id", "OwnerUserId", "Tags"]]

In [292]:
posts_poland

,Id,OwnerUserId,Tags
66,72,44.0,None
70,77,44.0,<air-travel><budget><online-resources><uk><air...
526,560,44.0,None
693,741,44.0,<luggage><children><airport-security>
850,921,355.0,<ferries><georgia-country><black-sea><ukraine>
...,...,...,...
117592,171514,9356.0,<where-on-earth>
117791,171751,89955.0,None
117899,171897,33002.0,None
118304,172396,27261.0,None


In [328]:
count_tags = copy.deepcopy(tags["TagName"]).reset_index()
count_tags["ctn"] = 0
count_tags
temp = count_tags["TagName"].apply(lambda x: posts_poland["Tags"].str.contains(x).sum()).reset_index().rename(columns={"TagName": "ctn"})

In [351]:
temp["ctn"].argmax()

34

In [353]:
tags.loc[34, :]

Id                       46
TagName          air-travel
Count                  4557
ExcerptPostId         255.0
WikiPostId            254.0
Name: 34, dtype: object

**Jakie słowo o długości powyżej 7 znaków pojawia się najczęściej w treści (Body) postów?**

In [8]:
posts_count_words = copy.deepcopy(posts.loc[:, ["Body"]])

In [33]:
good_posts = (
  posts['Body']
  .str.replace(pat="html[s]?S+", repl=" ", regex=True)
  .str.replace(pat="&quot;", repl=" ", regex=True)
  .str.replace(pat="&[0-9#]*;", repl=" ", regex=True)
  .str.replace(pat='<.*?>', repl=" ", regex=True)
  .str.replace(pat="\\n",repl=" ", regex=True)
  .str.replace(pat="[.,;><%/?\(\)”“\"\[\]` #]", repl=" ", regex=True)
  .str.lower()
  .str.split(" ")
  .dropna()
  .apply(lambda x: list(filter(lambda y: len(y)>7, x)))
  .explode()
  .value_counts()
)

In [34]:
good_posts

passport          36783
schengen          18030
countries         15418
application       14493
immigration       13941
                  ...  
slower-moving         1
26-week-long          1
sacombank             1
localscope            1
not-for-income        1
Name: Body, Length: 103202, dtype: int64

In [36]:
en_wiki.loc[en_wiki["curr"] == "Passport"]['n'].sum()

31631